# sphinx visualization demo

本demo用于直观调试数据预处理流程和模型inference流程

进行模型训练和生成测试结果请见launcher.py

#### 使用说明

1. 先执行以下2个cell以及初始化部分的3个cell
2. 后续三个测试部分可以选择运行，不必按顺序执行
3. 每个测试部分可以多次执行

In [ ]:
import os

import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

import utils as ut
from config import Config
from datagen import DataGenerator, _relative_points, _pad_img
from model import SphinxModel
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
def predict_one(model, images, general_category, new_p, weight, orig_size, image_visual):
    """
    params
         model: inference model
         images: padded and resized image, shape [1, img_size, img_size]
         general_category: 'top' or 'bottom'
         new_p: padded ground truth, shape [num_points, 2]
         weight: point weight, shape [num_points]
         orig_size: raw image size
         image_visual: padded raw image, shape [1, orig_size, orig_size]
    """
    # get prediction
    pred = model.Session.run(model.output[3], feed_dict={model.img: images})
    pred = pred[0]

    # calculate predictions from heatmap
    pred_idxes = np.zeros((model.num_points, 2), dtype=np.float32)
    total_dist = 0.0
    for j in range(model.num_points):
        if weight[j] == 1:
            index = np.unravel_index(pred[:, :, j].argmax(), (model.out_size, model.out_size))
            index = (index[1], index[0])
            pred_idx = np.array(index) / model.out_size * orig_size
            pred_idxes[j, :] = pred_idx
            total_dist += np.linalg.norm(pred_idx - new_p[j])

    # visualize prediction
    pred_hm_visual = dataset._generate_hm(orig_size, model.out_size, pred_idxes, weight, keep_invisible=False, sigma=24)
    pred_hm_visual = np.sum(pred_hm_visual, 2) * 255
    pred_hm_visual = cv2.resize(pred_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
    pred_hm_visual = np.expand_dims(pred_hm_visual, 2)
    image_visual = image_visual + pred_hm_visual.astype(np.int32)  
    plt.imshow(image_visual)
    plt.show()

    # calc normalized average error
    if general_category == 'top':
        norm_idx1 = new_p[5]
        norm_idx2 = new_p[6]
    else:
        norm_idx1 = new_p[0]
        norm_idx2 = new_p[1]

    norm_dist = np.linalg.norm(norm_idx2 - norm_idx1)
    error = total_dist / norm_dist / np.sum(np.count_nonzero(weight))
    print("image error:", error)

## 初始化数据集和模型

#### 注意修改以下值：
1. CUDA_VISIBLE_DEVICES
2. cfg.load

#### 关于初始化和硬件占用
1. 模型初始化后，关闭该页面不会终止显存占用，再次打开页面也不需要重新初始化
2. 如果要释放显存，需要在页面内restart kernel，或在jupyter首页shutdown该页面

In [ ]:
cfg = Config()

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cfg.load = "sphinx_20"

dataset = DataGenerator(cfg)
dataset.generate_set(train = True)
dataset.generate_set(train = False)

out_size = cfg.out_rate * cfg.hm_size

In [ ]:
model = SphinxModel(cfg, dataset)
model.generate_model(False)

In [ ]:
with tf.name_scope('Session'):
    model.Session = tf.Session()
    model._init_variable()

## 在训练集上进行测试
1. 主要是检查data augmentation的效果
2. 每次测试1张图片

In [ ]:
train_gen = dataset.generator(cfg.img_size, cfg.hm_size, 1, cfg.nStacks, 'train')

In [ ]:
# generate augmented data
img, hm0, hm1, hm2, hm3, weight = next(train_gen)
image = img[0]
hm = hm3[0]
weight = weight[0] 

raw_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
raw_image = cv2.resize(raw_image, (model.out_size, model.out_size), interpolation=cv2.INTER_LINEAR)

# visualize ground truth
gt_hm_visual = np.sum(hm, 2) * 255
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = raw_image + gt_hm_visual.astype(np.int32)
plt.imshow(gt_visual)
plt.show()

# calculate ground truth from gt_maps
new_p = np.zeros((model.num_points, 2))
for i in range(model.num_points):
    if weight[i] == 1:
        this_map = hm[:, :, i]
        pred_idx = np.reshape(np.array(np.array(np.where(this_map == np.max(this_map)))), [2])
        pred_idx[0], pred_idx[1] = pred_idx[1], pred_idx[0]
        new_p[i] = pred_idx

# visualize prediction
predict_one(model, img, 'bottom', new_p, weight, model.out_size, raw_image)

## 在验证集上进行测试

1. 每次测试1张图片。修改i控制图片位置
2. 为方便调试，这里不使用generator

In [ ]:
i = 0

In [ ]:
# get image info
name = dataset.valid_set[i]
point = dataset.data_dict[name]['points']
label = dataset.data_dict[name]['label']
weight = np.asarray(dataset.data_dict[name]['weight'])

# open image
img = dataset.open_img(dataset.train_img_dir, name)
orig_size = max(img.shape)

# pad image and points
new_p = _relative_points(point, img.shape)
img = _pad_img(img)

# visualize ground truth
gt_hm_visual = dataset._generate_hm(orig_size, cfg.img_size, new_p, weight, keep_invisible=False, sigma=12)
gt_hm_visual = np.sum(gt_hm_visual, 2) * 255
gt_hm_visual = cv2.resize(gt_hm_visual, (orig_size, orig_size), interpolation=cv2.INTER_LINEAR)
gt_hm_visual = np.expand_dims(gt_hm_visual, 2)
gt_visual = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) + gt_hm_visual.astype(np.int32)
plt.imshow(gt_visual)
plt.show()

pred_visual = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# resize image for inference
img = cv2.resize(img, (cfg.img_size, cfg.img_size), interpolation=cv2.INTER_LINEAR)
img = img.astype(np.float32) / 255
images = np.expand_dims(img, 0)

# visualize prediction
predict_one(model, images, 'bottom', new_p, weight, orig_size, pred_visual)
        
i += 1

## 在测试集上进行测试
1. 使用test_generator进行测试
2. 每次测试1张图片

In [ ]:
test_gen = dataset.test_generator(cfg.img_size, 1)

In [ ]:
# Traversal the test set
images, categories, offsets, names, sizes = next(test_gen)
prediction = model.Session.run(model.output[3], feed_dict={model.img: images})
hms = prediction

# init
pred_idxes = np.zeros((model.total_points, 2), dtype=np.float32)
img = images[0]
hm = hms[0]
offset = offsets[0]
category = categories[0]
name = names[0]
size = sizes[0]

# predict
write_line = [name, category]
cnt = 0
for j in range(model.total_points):
    if ut.VALID_POINTS[0][j]:
        if ut.VALID_POSITION[category][j] == 1:
            index = np.unravel_index(hm[:, :, cnt].argmax(), (model.out_size, model.out_size))
            index = (index[1], index[0])
            point = np.array(index) / model.out_size * size
            pred_idxes[j] = point
            point -= offset
            write_line.append(str(int(round(point[0]))) + '_' + str(int(round(point[1]))) + '_1')
        else:
            write_line.append('-1_-1_-1')
        cnt += 1
    else:
        write_line.append('-1_-1_-1')
print(write_line)
    
# visualize prediction
pred_hm_visual = np.zeros((size, size, 3))
pred_hm = dataset._generate_hm(size, size, pred_idxes, ut.VALID_POSITION[category], keep_invisible=False, sigma=12)
pred_hm = np.sum(pred_hm, 2) * 255
pred_hm_visual[:, :, 1] = pred_hm
pred_visual = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (size, size), interpolation=cv2.INTER_LINEAR)
pred_visual = pred_visual + pred_hm_visual.astype(np.int32)
# show results
plt.imshow(pred_visual)
plt.show()

In [ ]:
model._test()